##### I am Arsh Jaswal, 230205. This is my
# **Codeforces Problem and Solution Automated Extractor.**
In this, I have used undetected chromedriver, with selenium and beautifulsoup to get the desired result.
Also, **please refer to readme.md file before diving into the code.**

Key notes:
###### Used undetected chromedriver instead of selenium chromedriver
###### Considering only the codeforces website text editorials.
###### Not extracting 700 questions in one loop, using 7 loops to deal with browser failures
###### Using rating bracket of 1000-1200 

# **Step 1**

In [2]:
%pip install undetected-chromedriver  #NOT USING SELENIUM CHROME DRIVER BECAUSE IT WAS NOT BYPASSING HUMAN VERIFICATION
%pip install selenium

     ---------------------------------------- 0.0/65.4 kB ? eta -:--:--
     ----------- -------------------------- 20.5/65.4 kB 330.3 kB/s eta 0:00:01
     ----------------- -------------------- 30.7/65.4 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 65.4/65.4 kB 394.1 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for selenium>=4.9.0 from https://files.pythonhosted.org/packages/a6/1e/5f1a5dd2a28528c4b3ec6e076b58e4c035810c805328f9936123283ca14e/selenium-4.27.1-py3-none-any.whl.metadata
  Using cached selenium-4.27.1-py3-none-any.whl.metadata (7.1 kB)
  Obtaining dependency information for websockets from https://files.pythonhosted.org/packages/bc/ef/2d81679acbe7057ffe2308d422f744497b52009ea8bab34b6d74a2657d1d/websockets-14.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/3c/


## Installed undetected-chromedriver and selenium.
#### Not using selenium chromedriver, but can be used similarly
##### Reason: Human verification pops up due to Selenium chromedriver, so used undetected chromedriver, the same way
# **Step 2**
## Now importing libraries


In [162]:
from bs4 import BeautifulSoup
import json
import os
import time
import undetected_chromedriver as uc #Not using Selenium chromedriver because it was not bypassing human detector

import re #for pattern detection (tutorial, tutorial 1, tutorial #1, tut, editorial, editorial 1, edit, etc) but not
#video tutorial, vid tutorials, vid editorials, etc

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException




# **Step 3**
# CODEFORCES SCRAPPER FUNCTION
### A FUNCTION WHICH TAKES URL, DRIVER AND EXTRACTS THE DATA IN JSON FILE

#### 1. update json function which takes new data, and the key (contest number + problem part) and then adds it to json file

In [126]:
def update_json(new_data, contest_number,problem_part):
    if os.path.exists('ques_data.json'):
        with open('ques_data.json', 'r') as f:
            data = json.load(f)
    else:
        data = {}

    data[contest_number+problem_part] = new_data #adding new data to the json file

    with open('ques_data.json', 'w') as f:
        json.dump(data, f, indent=4)

#### 2. find solution function which takes editorial url, contest number, problem part, driver and problem url to find/trim the solution

In [134]:
def find_solution(url, driver, editorial_url, contest_number, problem_part):
    driver.get(editorial_url)

    #Delay after opening the editorial page
    time.sleep(2)

    WebDriverWait(driver, 6).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'ttypography'))
    )

    editorial_soup = BeautifulSoup(driver.page_source, "html.parser")


    #ISSUE: Different source code for different editorials
    # solutions=None
    # links=editorial_soup.find_all('a',href=True,title=True)
    # for link in links:
    #     href_value=link['href']
    #     if(href_value.endswith(f'/problem/{quePart}')):
    #         solution_class=link.find_next('div',class_="spoiler-content")
    #         code_sec=solution_class.find("code")
    #         if code_sec:
    #             solutions=code_sec.text
    
    #22.12.2024, THINKING TO PASS THE WHOLE TTYPOGRAPHY. It will include all answers of that contest though.
    #I can pass prompt to LLM to give only the {quePart} part from these solutions (Can fail, Todo tomorrow)

    #25.12.24 Or what I can do is, I can trim the solution such that, if let us say 2050 D is the problem part,
    #I can ask it to find 2050 D, then print everything till 2050 (D+1) => 2050 E (if E exists else till end)


    solutions = ""
    typography_elements = editorial_soup.find('div', class_='ttypography')

    if typography_elements:
        for element in typography_elements.find_all(['script', 'math']): #Done for formatting, we can anytime remove this, if we want math to be included
            element.decompose()

        solutions = typography_elements.get_text(strip=False)

    start = solutions.rfind(contest_number+problem_part)
    end = solutions.find(contest_number,start+1)


    if start!=-1:
        if end!=-1:
            specific_solution = solutions[start:end]
        else: #this is the last part
            specific_solution=solutions[start:]
    if start==-1:
        links=editorial_soup.find_all('a',href=True)
        for link in links:
            if link['href'].endswith(url.split("problem/")[-1]):
                nextp=link.find_next('p')
                if nextp:
                    specific_solution=nextp.get_text(strip=False)
    return specific_solution
    

#### 3. The parent function of the above two. It extracts all the problem data, as well as solution using find solution function and then updates json using update json function

In [161]:
def codeforces_scrapper(url,driver):
    try:
        driver.get(url)
        try:
            time.sleep(2) # delay added between requests
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'problem-statement'))
            )
        except TimeoutException:
            driver.refresh()  #one thing I encountered was that the problem url was getting stuck sometimes, so I added try expect to handle this, by refreshing if ever stucked
            time.sleep(2)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'problem-statement'))
            )

        soup = BeautifulSoup(driver.page_source, "html.parser")



        queTitle = soup.find('div', class_='title').text.strip()

        descParas = soup.find('div', class_='problem-statement').find_all('p')

        cleaned_desc_paras = [] #ISSUE: a and b => aa and bb, therefore decompositng when found element from math
        #Formatting here
        for para in descParas:
            for element in para.find_all([ 'script', 'math']):
                element.decompose()
            cleaned_desc_paras.append(para.get_text(strip=False)) #strip false because spaces will be cancelled, a and b=> aandb
        #formatting done
        desc = '\n'.join(cleaned_desc_paras)

        tags = [tag.text.strip() 
                    for tag in soup.find_all('span', class_='tag-box')]
        
        time_limit = soup.find(class_="time-limit").get_text(strip=True)

        memory_limit = soup.find(class_="memory-limit").get_text(strip=True)

        


        #finding contest number and the part
        parts = url.split('/')
        problem_part = parts[-1]
        contest_number = parts[-2] #2050C, use it to find in solution




        #*****************************ONLY TASK IS TO FIND SOLUTION NOW*******************************************
        #Shifting to editorial because done with problem part
        
        editorial_links=[]
        for a_tag in soup.find_all('a', {'href': True}): #should contain href
            title = a_tag.get('title', '').lower()
            link_text = a_tag.text.lower()
            
            if (re.search(r'(?<!video\s)(tut(orials?)?)', title, re.IGNORECASE) or re.search(r'(?<!video\s)(tut(orials?)?)', link_text, re.IGNORECASE) or
                re.search(r'(?<!video\s)(edit(orials?)?)', title, re.IGNORECASE) or re.search(r'(?<!video\s)(edit(orials?)?)', link_text, re.IGNORECASE)):
                editorial_links.append(a_tag['href'])
        
        
        specific_solution="No solution found"
        
        for editorial_link in editorial_links: #MAY BE TIME CONSUMING BUT EFFICIENT (CHECKS THE FIRST POSSIBLE TUTORIAL WHICH CAN BE EXTRACTED)
            editorial_url=f"https://codeforces.com{editorial_link}"
            if editorial_link.startswith("https://codeforces.com"): #Updating the Url because the link is the url
                editorial_url=editorial_link

            try:
                specific_solution=find_solution(url, driver, editorial_url, contest_number, problem_part)
                if specific_solution!="No solution found":
                    break  # SOlution found, break, we don't need to loop for something else

            except Exception as e:
                specific_solution="No solution found"

            time.sleep(2) #Request delays between editorials

            # print(desc)
            # print(specific_solution)
        new_data = {'title': queTitle,
                'description': desc,
                'time limit': time_limit,
                'memory limit': memory_limit,
                'tags': tags,
                'solutions': specific_solution}
            
        update_json(new_data, contest_number,problem_part)

    finally:
        pass


# **Step 4**
## MAIN FUNCTION

Function to call every problem from the problem set (defined as main function)


###### The below function is more specific one, where the range start and range end are fixed (rating range is fixed) and to change it, we have to change in the function. while the next one is more generic one, where we can change the range from outside the function. Also, the first function involves loop of 7 pages, which I have not run in final function call, as it may throw error due to such massive loop of 700 questions. In second function, I have not called loop, but worked over 1 page at a time only (i.e. 100 questions). Therefore calling 7 times it separately to avoid Chrome shutdown, or unexpected tab stuck.

## 1.

In [156]:
def main():
    driver = uc.Chrome()
    extractedProblems=set()
    try:
        rangestart=1000
        rangeend=1200
        for page in range(7): #Extracting 7 page:
            problemset_url=f"https://codeforces.com/problemset/page/{page+1}?tags={rangestart}-{rangeend}"
            driver.get(problemset_url)
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'problems'))
            )

            problemset_Soup=BeautifulSoup(driver.page_source, "html.parser")
            problem_links=problemset_Soup.find_all('a',href=True)
            
            for link in problem_links:
                if link['href'].startswith("/problemset/problem"):
                    problem_url=f"https://codeforces.com{link['href']}"
                    if problem_url in extractedProblems:
                        continue
                    print(problem_url)
                    driver.get(problem_url)
                    WebDriverWait(driver, 7).until(
                        EC.presence_of_element_located((By.CLASS_NAME, 'problem-statement'))
                    )
                    extractedProblems.add(problem_url)
                    codeforces_scrapper(problem_url,driver)
                    time.sleep(2)
    finally:
        driver.quit()

CAN BE DONE USING LOOP AS MENTIONED ABOVE
BUT DOING IT SEPARATELY SO THAT I DON'T ENCOUNTER ANY REFRESH GLITCH OR LOADING FAIL GLITCH DUE TO CONTINUOUS OPENING AND CLOSING OF TABS
EACH PAGE CONTAINS 100 QUESTIONS, WHICH IMPLIES CALLING THIS FUNCTION 7 TIMES TO GET 700 QUESTIONS OF RATING 1000-1200
7 PAGES (700 QUESTIONS) IF OPENED IN ONE LOOP (ONE GO) IT MAY THROW EXCEPTION ERROR, DUE TO OPENING OF 700*3=2100 URLS (ASSUMING EACH QUESTION CONTAIN ONLY 1 TUTORIAL, WHICH IS LEAST MINIMUM CASE)


## 2.

In [159]:

def main(rangestart,rangeend,page):
    driver=uc.Chrome()
    extractedProblems=set()
    try:
        problemset_url=f"https://codeforces.com/problemset/page/{page}?tags={rangestart}-{rangeend}"
        print(f"PAGE {page}")
        driver.get(problemset_url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'problems'))
        )

        problemset_Soup=BeautifulSoup(driver.page_source, "html.parser")
        problem_links=problemset_Soup.find_all('a',href=True)
        
        for link in problem_links:
            if link['href'].startswith("/problemset/problem"):
                problem_url=f"https://codeforces.com{link['href']}"
                if problem_url in extractedProblems:
                    continue
                print(problem_url)
                driver.get(problem_url)

                time.sleep(2) #Delay between requests, as mentioned

                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'problem-statement'))
                )
                extractedProblems.add(problem_url)
                codeforces_scrapper(problem_url,driver)
    finally:
        driver.quit()

# **Step 5**
### Choosing Rating Bracket

In [160]:
#can change rating range here
rating_start=1000
rating_end=1200

# **Step 6**
## Running the main file

#### Extracting from page 1, first 100 questions

In [95]:
if __name__ == "__main__":
    main(rating_start,rating_end,1) #not calling loop because of larger data set (can call loop, but that will lead to a massive dataset, therefore dividing it into 100*7)

PAGE 1
https://codeforces.com/problemset/problem/2050/C
https://codeforces.com/problemset/problem/2050/B
https://codeforces.com/problemset/problem/2048/C
https://codeforces.com/problemset/problem/2046/A
https://codeforces.com/problemset/problem/2044/D
https://codeforces.com/problemset/problem/2041/E
https://codeforces.com/problemset/problem/2041/B
https://codeforces.com/problemset/problem/2040/B
https://codeforces.com/problemset/problem/2039/C1
https://codeforces.com/problemset/problem/2039/B
https://codeforces.com/problemset/problem/2037/C
https://codeforces.com/problemset/problem/2036/C
https://codeforces.com/problemset/problem/2034/B
https://codeforces.com/problemset/problem/2032/B
https://codeforces.com/problemset/problem/2030/C
https://codeforces.com/problemset/problem/2029/B
https://codeforces.com/problemset/problem/2027/B
https://codeforces.com/problemset/problem/2025/B
https://codeforces.com/problemset/problem/2024/B
https://codeforces.com/problemset/problem/2021/B
https://code

#### Extracting from page 2 (100-200 questions). Also finding time per question query for better analysis.

In [99]:
if __name__ == "__main__":
    initialtime= time.time() # initial time
    main(rating_start,rating_end,2)
    finaltime=time.time()-initialtime
    print(f"Average time per question: {finaltime/100}")

PAGE 2
https://codeforces.com/problemset/problem/1914/D
https://codeforces.com/problemset/problem/1914/C
https://codeforces.com/problemset/problem/1913/B
https://codeforces.com/problemset/problem/1910/A
https://codeforces.com/problemset/problem/1909/B
https://codeforces.com/problemset/problem/1907/C
https://codeforces.com/problemset/problem/1907/B
https://codeforces.com/problemset/problem/1906/A
https://codeforces.com/problemset/problem/1905/B
https://codeforces.com/problemset/problem/1904/B
https://codeforces.com/problemset/problem/1903/B
https://codeforces.com/problemset/problem/1902/B
https://codeforces.com/problemset/problem/1901/B
https://codeforces.com/problemset/problem/1899/C
https://codeforces.com/problemset/problem/1899/B
https://codeforces.com/problemset/problem/1894/B
https://codeforces.com/problemset/problem/1891/B
https://codeforces.com/problemset/problem/1886/B
https://codeforces.com/problemset/problem/1884/B
https://codeforces.com/problemset/problem/1883/C
https://codef

### Average time per question = 8.2282 sec
###### Minimum 3 different webpages are opened (one problemset, oe problem, atleast one of the n editorials), Considering webpage loading time = 2 sec for each. 8.22 sec is pretty reasonable time

#### Similarly Page 3 (200-300)

In [102]:
if __name__ == "__main__":
    main(rating_start,rating_end,3)

PAGE 3
https://codeforces.com/problemset/problem/1784/A
https://codeforces.com/problemset/problem/1783/B
https://codeforces.com/problemset/problem/1781/B
https://codeforces.com/problemset/problem/1780/B
https://codeforces.com/problemset/problem/1772/C
https://codeforces.com/problemset/problem/1770/B
https://codeforces.com/problemset/problem/1770/A
https://codeforces.com/problemset/problem/1769/C1
https://codeforces.com/problemset/problem/1769/B1
https://codeforces.com/problemset/problem/1766/B
https://codeforces.com/problemset/problem/1765/M
https://codeforces.com/problemset/problem/1765/E
https://codeforces.com/problemset/problem/1763/B
https://codeforces.com/problemset/problem/1762/B
https://codeforces.com/problemset/problem/1761/B
https://codeforces.com/problemset/problem/1760/E
https://codeforces.com/problemset/problem/1760/D
https://codeforces.com/problemset/problem/1759/C
https://codeforces.com/problemset/problem/1747/C
https://codeforces.com/problemset/problem/1744/D
https://cod

#### Page 4 (300-400)

In [103]:
if __name__ == "__main__":
    main(rating_start,rating_end,4)

PAGE 4
https://codeforces.com/problemset/problem/1616/B
https://codeforces.com/problemset/problem/1614/B
https://codeforces.com/problemset/problem/1613/C
https://codeforces.com/problemset/problem/1613/B
https://codeforces.com/problemset/problem/1611/C
https://codeforces.com/problemset/problem/1610/B
https://codeforces.com/problemset/problem/1609/B
https://codeforces.com/problemset/problem/1608/B
https://codeforces.com/problemset/problem/1607/C
https://codeforces.com/problemset/problem/1606/B
https://codeforces.com/problemset/problem/1605/B
https://codeforces.com/problemset/problem/1604/B
https://codeforces.com/problemset/problem/1602/B
https://codeforces.com/problemset/problem/1598/C
https://codeforces.com/problemset/problem/1598/B
https://codeforces.com/problemset/problem/1594/C
https://codeforces.com/problemset/problem/1594/B
https://codeforces.com/problemset/problem/1593/D1
https://codeforces.com/problemset/problem/1593/C
https://codeforces.com/problemset/problem/1592/B
https://code

In [145]:
#DEBUGGING THE CASE WHEN EDITORIAL LINK IS https://codeforces.com/blog/... instead of /blog/...

# driver=uc.Chrome()
# url="https://codeforces.com/problemset/problem/1263/A"
# codeforces_scrapper(url,driver)

#SUCESS

#### Page 5 (500-600)

In [146]:
if __name__ == "__main__":
    main(rating_start,rating_end,5)

PAGE 5
https://codeforces.com/problemset/problem/1426/C
https://codeforces.com/problemset/problem/1421/B
https://codeforces.com/problemset/problem/1420/B
https://codeforces.com/problemset/problem/1419/D1
https://codeforces.com/problemset/problem/1419/B
https://codeforces.com/problemset/problem/1418/A
https://codeforces.com/problemset/problem/1417/B
https://codeforces.com/problemset/problem/1415/B
https://codeforces.com/problemset/problem/1413/B
https://codeforces.com/problemset/problem/1411/B
https://codeforces.com/problemset/problem/1409/C
https://codeforces.com/problemset/problem/1409/B
https://codeforces.com/problemset/problem/1407/A
https://codeforces.com/problemset/problem/1406/B
https://codeforces.com/problemset/problem/1405/B
https://codeforces.com/problemset/problem/1401/B
https://codeforces.com/problemset/problem/1399/C
https://codeforces.com/problemset/problem/1395/B
https://codeforces.com/problemset/problem/1395/A
https://codeforces.com/problemset/problem/1392/C
https://code

#### Page 6 (500-600)

In [152]:
if __name__ == "__main__":
    main(rating_start,rating_end,6)

PAGE 6
https://codeforces.com/problemset/problem/1237/A
https://codeforces.com/problemset/problem/1234/B1
https://codeforces.com/problemset/problem/1231/C
https://codeforces.com/problemset/problem/1230/B
https://codeforces.com/problemset/problem/1227/B
https://codeforces.com/problemset/problem/1227/A
https://codeforces.com/problemset/problem/1225/B1
https://codeforces.com/problemset/problem/1223/B
https://codeforces.com/problemset/problem/1221/C
https://codeforces.com/problemset/problem/1221/B
https://codeforces.com/problemset/problem/1221/A
https://codeforces.com/problemset/problem/1215/A
https://codeforces.com/problemset/problem/1214/C
https://codeforces.com/problemset/problem/1214/B
https://codeforces.com/problemset/problem/1213/C
https://codeforces.com/problemset/problem/1213/B
https://codeforces.com/problemset/problem/1211/A
https://codeforces.com/problemset/problem/1207/B
https://codeforces.com/problemset/problem/1205/A
https://codeforces.com/problemset/problem/1204/A
https://cod

### Average time per question for 6th page = 4.904 sec

#### Page 7 (600-700)

In [153]:
if __name__ == "__main__":
    main(rating_start,rating_end,7)

PAGE 7
https://codeforces.com/problemset/problem/1073/A
https://codeforces.com/problemset/problem/1070/K
https://codeforces.com/problemset/problem/1068/B
https://codeforces.com/problemset/problem/1066/A
https://codeforces.com/problemset/problem/1064/B
https://codeforces.com/problemset/problem/1060/B
https://codeforces.com/problemset/problem/1059/A
https://codeforces.com/problemset/problem/1054/B
https://codeforces.com/problemset/problem/1051/B
https://codeforces.com/problemset/problem/1051/A
https://codeforces.com/problemset/problem/1043/B
https://codeforces.com/problemset/problem/1042/B
https://codeforces.com/problemset/problem/1042/A
https://codeforces.com/problemset/problem/1041/B
https://codeforces.com/problemset/problem/1040/A
https://codeforces.com/problemset/problem/1038/B
https://codeforces.com/problemset/problem/1036/A
https://codeforces.com/problemset/problem/1033/B
https://codeforces.com/problemset/problem/1033/A
https://codeforces.com/problemset/problem/1032/B
https://codef

# **Step 7**
### Making text file of problem statements

I didn't make text file of problem statements above. But, We can Also make this txt file while making Json file (i.e. both in one loop), but making it from already made json file. Since I don't want to hamper my results again by re-looping the database of 700 questions

In [169]:
def save_problem_statements_from_json(json_file):
    folder = "problem_statements"
    os.makedirs(folder, exist_ok=True)

    if os.path.exists(json_file):
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        print(f"typo in Json file")
        return
    
    for key, problem in data.items():
        title = problem.get('title', "No title found")
        description = problem.get('description', "No description found.")
        constraints=[]
        constraints.append(problem.get('time limit',"no time limit found"))
        constraints.append(problem.get('memory limit',"no memory limit found"))
        constraints='\n'.join(constraints) #overwriting

        safe_title = key #contains only contextnumber and problem part
        filename = os.path.join(folder, f"{safe_title}.txt")
        

        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"Problem Title: {title}\n\n{constraints}\n\n{description}")
        
        print(f"Saved: {filename}")

save_problem_statements_from_json("ques_data.json")
#we can use this function in codescrapper with few changes


Saved: problem_statements\265B.txt
Saved: problem_statements\482A.txt
Saved: problem_statements\265A.txt
Saved: problem_statements\2050C.txt
Saved: problem_statements\2050B.txt
Saved: problem_statements\2048C.txt
Saved: problem_statements\2046A.txt
Saved: problem_statements\2044D.txt
Saved: problem_statements\2041E.txt
Saved: problem_statements\2041B.txt
Saved: problem_statements\2040B.txt
Saved: problem_statements\2039C1.txt
Saved: problem_statements\2039B.txt
Saved: problem_statements\2037C.txt
Saved: problem_statements\2036C.txt
Saved: problem_statements\2034B.txt
Saved: problem_statements\2032B.txt
Saved: problem_statements\2030C.txt
Saved: problem_statements\2029B.txt
Saved: problem_statements\2027B.txt
Saved: problem_statements\2025B.txt
Saved: problem_statements\2024B.txt
Saved: problem_statements\2021B.txt
Saved: problem_statements\2020B.txt
Saved: problem_statements\2019B.txt
Saved: problem_statements\2014C.txt
Saved: problem_statements\2009C.txt
Saved: problem_statements\2008

In [165]:
# print(os.getcwd())

c:\Users\arshc\Desktop\Python


In [ ]:
#DEBUGGING
# url = "https://codeforces.com/problemset/problem/265/A"
# codeforces_scrapper(url) 

In this problem you just need to implement what is written in the statement. Make a variable that holds the position of Liss, and simulate the instructions one by one.


# Thank you :)
You can refer to the **json file.**
89/700 cases of "No solution found". 611 solution found for 700 queries (87.28%).